## 1. Extraccion de Datos

### 1.1 Obtencion de datos Proprocesados

In [0]:
import pandas as pd
from io import BytesIO
from minio import Minio
import urllib3
# Establecer el número máximo de columnas a mostrar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Conectar al servidor de MinIO
client = Minio(
    #"minio.minio-user.svc.cluster.local:80",  # Dirección del servidor MinIO
    "192.168.25.223",  # Dirección del servidor MinIO
    access_key="c9iXL6uoEu8r35odfMLV",         # Clave de acceso
    secret_key="r3Wx21EmA41gB3mH65mvBVG9sH3lIMPwSmD0WMtI",  # Clave secreta
    secure=False,  # Cambia a True si usas HTTPS
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE')  # Ignora la verificación del certificado
)

# Nombre del bucket y del archivo
bucket_name = 'hudbayprocessed'
object_name = 'proyectocombustible_processed/table_delta_28.parquet'  # Nombre del archivo Parquet que quieres leer

# Obtener el archivo Parquet del bucket
response = client.get_object(bucket_name, object_name)
data = response.read()

# Convertir los datos leídos en un DataFrame de Pandas
parquet_buffer = BytesIO(data)
datos = pd.read_parquet(parquet_buffer)

# Mostrar el DataFrame
datos.head()

In [0]:
import pyarrow.parquet as pq
import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import pandas as pd

# Configurar pandas para que muestre todas las filas y columnas
pd.set_option('display.max_rows', None)  # Sin límite en el número de filas
pd.set_option('display.max_columns', None)  # Sin límite en el número de columnas

ruta_carpeta_parquet = "/dbfs/mnt/datalakemlopsd4m/processed/proyectocombustible_processed/hudbay/table_delta_28.parquet"
tabla = pq.read_table(ruta_carpeta_parquet)

# Convertir a DataFrame de pandas y mostrar las primeras filas
datos = tabla.to_pandas()
datos.head(1)

### 1.2 Revision de los datos Extraidos

In [0]:
datos.shape

Verificar si hay valores Nulos

In [0]:
datos.isnull().sum()

In [0]:
#Ver la fecha minima y maxima
datos['instant_date_t'].min(), datos['instant_date_t'].max(),

In [0]:
#ve los nombres de las columnas
datos.columns

## 2.  Preparacion de datos para Analitica Predictiva

### 2. Limpieza de valores Nulos

In [0]:
# 3.1 Supongamos que tienes un DataFrame llamado datos
valores_nulos = datos.isnull().sum()
valores_nulos_ordenados = valores_nulos.sort_values(ascending=False)
porcentaje_nulos = (valores_nulos_ordenados / len(datos)) * 100
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > 40].index
datos = datos.drop(columnas_a_eliminar, axis=1)

In [0]:
porcentaje_nulos

In [0]:
columnas_a_eliminar

In [0]:
datos.shape

In [0]:
datos.dtypes

In [0]:
#4.1 Eliminando columnas especificas que no aportan informacion ( # errors='ignore':ignore cualquier error si alguna de las columnas especificadas no se encuentra en el DataFrame.)
datos = datos.drop(['equipment','nombre'], axis=1, errors='ignore') 

In [0]:
datos.shape

In [0]:
# 3.4 Rellenar los valores nulos con ceros en todo el DataFrame
datos = datos.fillna(0)

In [0]:
datos.isnull().sum()

## 2.1 Tratamiento de Variables Categoricas

### 2.1.1 Metodo 1 - Usar un bucle para realizar el codigo de One Hot Encoding

2.1.1.1 VARIABLE Status Truck

In [0]:
# import pandas as pd
# # Obtener las categorías únicas
# categorias = datos['Status Truck'].unique()

# # Crear un DataFrame vacío con columnas para cada categoría, Diccionario one_hot_dict : categoria: clave es una categoría única de #nombre_equipo_carguio, y el [0]*len(datos) : valor asociado a cada clave es una lista de ceros de la misma longitud que el DataFrame 
# one_hot_dict = {categoria: [0]*len(datos) for categoria in categorias}

# # Llenar el diccionario con 1s en las posiciones correspondientes
# # idx es el índice de la fila, val es el valor de nombre_equipo_carguio en esa fila
# # Si idx=0 y val='CF02', entonces one_hot_dict['CF02'][0] = 1
# for idx, val in datos['Status Truck'].items():
#     one_hot_dict[val][idx] = 1

# # Convertir el diccionario a un DataFrame
# one_hot_df = pd.DataFrame(one_hot_dict)

# #Unir el DataFrame original con el codificado
# datos = pd.concat([datos, one_hot_df], axis=1)

# # # Mostrar el resultado
# datos[['Status Truck','Cargando', 'Viaje Descargado', 'Viaje Cargado', 'Descarga']].head()

### 2.2 Convertir la columna de tiempo a índice temporal si aún no lo es y Definimos Year, Month, week apartir de la fecha

In [0]:
import pandas as pd

# 1.Convertir la columna de tiempo a índice temporal si aún no lo es
datos.set_index('instant_date_t', inplace=True)

# #2. Ordenamos el dataset de forma ascendente segun el datetime
datos.sort_index(inplace=True)

# 3. Crear nuevas columnas basadas en el índice temporal
datos['year'] = datos.index.year  # year
datos['month'] = datos.index.month # mes
datos['day'] = datos.index.day  # Día del mes (1 al 31 del mes)
datos['hour'] = datos.index.hour  # hora
#datos['day_week_str'] = datos.index.strftime('%a')  # Día de la semana como string
#datos['year_month'] = datos.index.strftime('%Y_%m')  # Formato 'Año_Mes'
datos.head()
# (Opcional) Si quieres devolver 'instant_date_t' como columna
#datos = datos.reset_index()

In [0]:
#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()
df_time_diffs.value_counts()

## 3 Filtro de datos a usar: Equipo de Acarreo a Analizar

In [0]:
# datos_equipo_28 = datos[datos['eq_id'] == 28]

## 4. Seleccion de Variables para Modelos de Machine Learning

### 4.1 Seleccion variables con StatModels, para regresion lineal (variables significativas estadisticamente basado en la Hipotesis Nula)

In [0]:
datos.columns.values

In [0]:
import pandas as pd
import statsmodels.api as sm

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
# X = datos[['x', 'y', 'z', 
#         #'eq_id', 
#        'id_trabaj_t',
#        'tramosidsnew_t',
#         'Left Rear Axle Rpm',
#        'Right Rear Axle Rpm', 'Left Exhaust Temperature',
#        'Right Exhaust Temperature', 'Left Rear Parking Brake Oil Pressure',
#        'Right Rear Parking Brake Oil Pressure', 'Parking Brake',
#        'Brake Pump Pressure', 'Engine Oil Pressure', 'Actual Gear',
#        'Ground Speed', 'Engine Speed', 'Transmission Input Speed',
#        'Shift Lever Position', 'Throttle Position',
#        'Percent Engine Load At Current Engine Speed', 'Truck Payload',
#        'Transmission Output Speed', 'Transmission Output Speed #1',
#        'Transmission Output Speed #2', 'Air Filter #1 Restriction ',
#        'Air Filter #4 Restriction ', 'Air Filter #3 Restriction',
#        'Air Filter #2 Restriction ', 'Tank Fuel Level',
#        'Percent Fuel Position', 'Fuel Rail Pressure',
#        'Cargando', 'Viaje Descargado', 'Viaje Cargado', 'Descarga']]  # Reemplaza 'feature1', 'feature2', 'feature3' con tus nombres de características
X =  datos[datos.columns]

y = datos['Fuel Consumption Rate'] # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo

# Añadir constante a las características (intercepto)
X = sm.add_constant(X)

# Crear el modelo de regresión lineal con statsmodels
modelo = sm.OLS(y, X).fit()

# Imprimir un resumen estadistico del modelo
print(modelo.summary())

In [0]:
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [0]:
results_summary_to_dataframe(modelo)

### 4.2 Ver Correlaciones entre mis variables

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'

# Lista de variables que deseas analizar
variables_a_analizar = ['Engine Coolant Temperature', 'Left Exhaust Temperature',
       'Final Drive Oil Bypass Solenoid Status',
       'Rear Axle Pump Drive Oil Divert Solenoid Status',
       'Right Rear Axle Rpm', 'Fuel Pressure', 'Engine Speed',
       'Brake Pump Unloader Solenoid Status', 'Shift Lever Position',
       'Steering Accumulator Oil Pressure',
       'Left Front Brake Oil Temperature', 'Truck Payload Cycle Fela',
       'Turbocharger #3 Compressor Inlet Pressure (Absolute)',
       'Service Brake', 'Fuel Rail Temperature',
       'Desired Fuel Rail Pressure', 'Desired Brake Cooling Pump Speed',
       'Differential (Axle) Lube Pressure', 'Diff Oil Temperature',
       'Altitude Derate', 'Auto Lube Relay Status', 'Auto Retarder Mode',
       'Fuel Actuator Position Command', 'Left Rear Strut Pressure',
       'Transmission Oil Level Status', 'Air Filter #3 Restriction',
       'Left Rear Parking Brake Oil Pressure',
       'Steering Fluid Temperature', 'Engine Power Derate',
       'Transmission Lockout Status', 'Engine Pre-Lube',
       'Right Front To Left Front Strut Differential Pressure',
       'Engine Coolant Pump Outlet Temperature', 'Engine Derate',
       'Ambient Air Temperature', 'Right Rear Brake Oil Temperature',
       'Right Rear Parking Brake Oil Pressure', 'Fuel Consumption Rate']

# Filtrar el DataFrame para incluir solo las variables de interés
datos_subset = datos[variables_a_analizar]

# Crear una matriz de correlación para el subconjunto de variables
matriz_correlacion = datos_subset.corr()

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# Aumentar el tamaño del gráfico para mostrar más detalles
plt.figure(figsize=(25, 8))  #ancho * altura

# Crear un mapa de calor (heatmap) de la matriz de correlación con desplazamiento habilitado
ax = sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)

# Agregar etiquetas a los ejes X e Y
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right', fontsize=8)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje x)
ax.tick_params(axis='x', labelsize=12)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje y)
ax.tick_params(axis='y', labelsize=12)

plt.title("Matriz de Correlación Variables")
plt.show()

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'

# Lista de variables que deseas analizar
variables_a_analizar = [
        'Left Exhaust Temperature',
        'Right Exhaust Temperature',  
        'Parking Brake',
        'Engine Speed', 
        'Shift Lever Position', 'Throttle Position',
        'Percent Engine Load At Current Engine Speed',
         'Air Filter #1 Restriction ',
         'Air Filter #4 Restriction ', 'Air Filter #3 Restriction',
         'Air Filter #2 Restriction ', 
        'Percent Fuel Position', 'Fuel Rail Pressure','Fuel Consumption Rate']

# Filtrar el DataFrame para incluir solo las variables de interés
datos_subset = datos[variables_a_analizar]

# Crear una matriz de correlación para el subconjunto de variables
matriz_correlacion = datos_subset.corr()

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# Aumentar el tamaño del gráfico para mostrar más detalles
plt.figure(figsize=(25, 8))  #ancho * altura

# Crear un mapa de calor (heatmap) de la matriz de correlación con desplazamiento habilitado
ax = sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)

# Agregar etiquetas a los ejes X e Y
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right', fontsize=8)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje x)
ax.tick_params(axis='x', labelsize=12)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje y)
ax.tick_params(axis='y', labelsize=12)

plt.title("Matriz de Correlación Variables")
plt.show()

### 4.3 Ver la varianza de mis variables

In [0]:
datos_varianza=datos[['Engine Coolant Temperature', 'Left Exhaust Temperature',
       'Final Drive Oil Bypass Solenoid Status',
       'Rear Axle Pump Drive Oil Divert Solenoid Status',
       'Right Rear Axle Rpm', 'Fuel Pressure', 'Engine Speed',
       'Brake Pump Unloader Solenoid Status', 'Shift Lever Position',
       'Steering Accumulator Oil Pressure',
       'Left Front Brake Oil Temperature', 'Truck Payload Cycle Fela',
       'Turbocharger #3 Compressor Inlet Pressure (Absolute)',
       'Service Brake', 'Fuel Rail Temperature',
       'Desired Fuel Rail Pressure', 'Desired Brake Cooling Pump Speed',
       'Differential (Axle) Lube Pressure', 'Diff Oil Temperature',
       'Altitude Derate', 'Auto Lube Relay Status', 'Auto Retarder Mode',
       'Fuel Actuator Position Command', 'Left Rear Strut Pressure',
       'Transmission Oil Level Status', 'Air Filter #3 Restriction',
       'Left Rear Parking Brake Oil Pressure',
       'Steering Fluid Temperature', 'Engine Power Derate',
       'Transmission Lockout Status', 'Engine Pre-Lube',
       'Right Front To Left Front Strut Differential Pressure',
       'Engine Coolant Pump Outlet Temperature', 'Engine Derate',
       'Ambient Air Temperature', 'Right Rear Brake Oil Temperature',
       'Right Rear Parking Brake Oil Pressure']]

In [0]:
desviacion_estandar_sorted

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calcular la varianza de cada columna
desviacion_estandar = np.sqrt(datos_varianza.var())  #desviacion estandar 

# Redondear la varianza a 2 decimales
#variance_rounded = variance.round(2)

# Ordenar las varianzas de mayor a menor
desviacion_estandar_sorted = desviacion_estandar.sort_values(ascending=False)

# Graficar los resultados
plt.figure(figsize=(10, 6))
desviacion_estandar_sorted.plot(kind='bar')
plt.title('Varianza de Variables del DataFrame')
plt.xlabel('Variables')
plt.ylabel('Varianza')
# plt.xticks(rotation=45)
plt.show()

Variables clave para incluir en el modelo serían:
- Fuel Rail Pressure (Presión del riel de combustible)
- Truck Payload (Carga del camión)
- Engine Speed (Velocidad del motor)
- Throttle Position (Posición del acelerador)
- Percent Engine Load (Carga del motor al % de la velocidad actual)
- Ground Speed (Velocidad en tierra)
- Percent Fuel Position (porcentaje de la inyección de combustible)
- Tank Fuel Level (Nivel del tanque de combustible) **

## 5. Analisis de Series Temporales

#### 5.1 Serie Temporal con datos Originales

In [0]:
import matplotlib.pyplot as plt
 
# Set pyplot style
# Plot
datos['Fuel Consumption Rate'].plot(title='Consumo Combustible - Time Plot', figsize=(10,6)) # datos['Fuel Consumption Rate'],indica que los valores de esta columna irán en el eje Y, el eje X considera el index
plt.ylabel('Consumo Combustible (Galones)')   # X está implícito y toma los valores del índice del DataFrame
plt.xlabel('Date')

#### 5.2 Gráfico de líneas donde el eje X representa las fechas y el eje Y el consumo de combustible en galones

Metodos para Eliminar Ruido de Datos

1. Suavizado con Media Móvil

Por ejemplo si tenemos un consumo por dia, Consumo Diario = [10, 12, 14, 13, 15, 17, 20, 18, 16, 14]
- Y decides usar una ventana de 3 días para el suavizado.
- Resultados:

El primer valor de la serie suavizada será 12 (promedio de los primeros 3 días).
El segundo valor será 13 (promedio de los días 2, 3 y 4).
Y así sucesivamente.

In [0]:
import plotly.graph_objs as go
import plotly.express as px

# Suavizado con media móvil
window_size = 30  # Tamaño de la ventana de la media móvil (puedes ajustar esto según tus datos)
datos['Smoothed Fuel Consumption Rate'] = datos['Fuel Consumption Rate'].rolling(window=window_size).mean()

# Crear la figura interactiva con Plotly
fig = go.Figure()

# Agregar la línea del consumo de combustible suavizado
fig.add_trace(go.Scatter(
    x=datos.index,  # Eje X: Fechas
    y=datos['Smoothed Fuel Consumption Rate'],  # Eje Y: Consumo suavizado
    mode='lines',
    name='Media Móvil',
    line=dict(color='orange')
))

# Crear los ticks manualmente cada 3 días
tick_values = pd.date_range(start=datos.index.min(), end=datos.index.max(), freq='2D')

# Configurar los títulos y etiquetas de los ejes
fig.update_layout(
    title='Consumo Combustible - Time Plot',
    xaxis_title='Fecha',
    yaxis_title='Consumo Combustible (Galones)',
    xaxis=dict(
        tickformat='%Y-%m-%d',  # Formato para las fechas
        tickangle=45,  # Rotar etiquetas del eje X
        tickvals=tick_values,  # Colocar etiquetas cada 3 días
        #dtick='D3'  # Mostrar cada 7 días en el eje X
    ),
    autosize=False,
    width=1000,
    height=600
)

# Mostrar el gráfico interactivo
fig.show()

### 5.3 Analisis de Tendencia, Estacionalidad, Residuo

In [0]:
# # Interpolación para llenar valores faltantes
# datos_interpolados = datos_reindexados.interpolate(method='linear')

# # Alternativamente, puedes usar el método de llenado hacia adelante o hacia atrás
# # datos_interpolados = datos_reindexados.fillna(method='ffill')  # Llenar hacia adelante
# # datos_interpolados = datos_reindexados.fillna(method='bfill')  # Llenar hacia atrás

# # Mostrar las primeras filas del DataFrame con interpolación
# print(datos_interpolados.head())

In [0]:
# # Eliminar duplicados conservando solo la primera ocurrencia
# datos_equipo_28_enero_sin_duplicados = datos_equipo_28_enero[~datos_equipo_28_enero.index.duplicated(keep='first')]

# # Mostrar las primeras filas del DataFrame limpio
# datos_equipo_28_enero_sin_duplicados.head()

In [0]:
datos.index

In [0]:
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt


# Asegúrate de que el índice es de tipo DatetimeIndex
# datos_equipo_32.index = pd.to_datetime(datos_equipo_32.index)

window_size = 15

# Define el período de estacionalidad (por ejemplo, si los datos son diarios y la estacionalidad es semanal, el período sería 7)
seasonal_period = 2 # Ajusta este valor según la estacionalidad de tus datos

# Descomposición STL
stl = STL(datos['Fuel Consumption Rate'], period=seasonal_period, seasonal=window_size)
result = stl.fit()

# Graficar la tendencia, estacionalidad y residuos
plt.figure(figsize=(12,8))
plt.subplot(3,1,1)
plt.plot(result.trend, label='Tendencia')
plt.legend()
plt.subplot(3,1,2)
plt.plot(result.seasonal, label='Estacionalidad')
plt.legend()
plt.subplot(3,1,3)
plt.plot(result.resid, label='Residuo')
plt.legend()
plt.tight_layout()
plt.show()

### 5.4 Analisis Autoregresivo de la Serie Temporal

In [0]:
from statsmodels.graphics.tsaplots import plot_acf
# Hacer un gráfico de autocorrelación para la serie de consumo de combustible
plot_acf(datos['Fuel Consumption Rate'], lags=800)  # 'lags=40' son los retardos
plt.title('Autocorrelación del Consumo de Combustible')
plt.show()

### Seleccion de Variables para Modelo IA

In [0]:
datos.columns

In [0]:
#En Multivariado, poner la variable a predecir, en la columna final
c_variables = ['Engine Coolant Temperature', 'Left Exhaust Temperature','Fuel Consumption Rate']


datos_df = datos[c_variables]  # Modelo Multivariado
datos_df.head()  

### 3. Escalar/Normalizar los datos
Es requerido para garantizar que todas las caracteristicas se encuentren en el mismo rango de valores, lo que facilita el entrenamiento del Modelo y las predicciones

Importante para escalar: Filtrar solo las columnas numéricas y preferible no considerar la variable TARGET

In [0]:
import numpy as np

def escalar_por_variables_especificas(dataframe, variables_a_escalar, medias, desviaciones_estandar):
  """
  Escala solo las variables especificadas en un dataframe de forma independiente.

  Args:
    dataframe: El dataframe que se va a escalar.
    variables_a_escalar: Una lista que contiene los nombres de las variables que se van a escalar.
    medias: Un diccionario que contiene la media de cada variable.
    desviaciones_estandar: Un diccionario que contiene la desviación estándar de cada variable.

  Returns:
    El dataframe escalado.
  """
  dataframe_escalado = dataframe.copy()
  for columna in dataframe.columns:
    if columna in variables_a_escalar:
      dataframe_escalado[columna] = (dataframe[columna] - medias[columna]) / desviaciones_estandar[columna]
  return dataframe_escalado


# Seleccionar todas las columnas excepto la variable objetivo
datos_x_escalar = datos_df.drop(columns=['Fuel Consumption Rate'])

variables_a_escalar = datos_x_escalar.columns

# Calcular medias y desviaciones estándar para todas las variables
medias = datos_x_escalar.mean()
desviaciones_estandar = datos_x_escalar.std()

# Escalar solo las variables especificadas en el dataframe train
df_escalado = escalar_por_variables_especificas(datos_x_escalar, variables_a_escalar, medias, desviaciones_estandar)

In [0]:
df_escalado.head()

### Armamos y Unimos los Datos Escalados con Target Original

In [0]:
# Identificamos el target del data original
target_ml = datos_df['Fuel Consumption Rate']

#Agregamos la target a nuestro datos escalados previamente
df_escalado['Fuel Consumption Rate'] = target_ml

In [0]:
df_escalado.head()

### Pasar los datos de serie temporal univariada o multivariada como un conjunto de datos de aprendizaje supervisado

#### OPCION 1: Usando una funcion de restraso(lags) y adelantos(pasos al futuro)

In [0]:
# Función de Python llamada series_to_supervised() que toma una serie temporal univariada o multivariada y la encuadra como un #conjunto de datos de aprendizaje supervisado.
import pandas as pd
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Secuencia de observaciones como una lista o matriz NumPy 2D.
    n_in : número de observaciones de retraso como entrada ( X ). Los valores pueden estar entre [1..len(data)] Opcional. El valor predeterminado es 1.
    n_out : Número de observaciones como salida ( y ). Los valores pueden estar entre [0..len(data)-1]. Opcional. El valor predeterminado es 1.
    dropnan : valor booleano para eliminar o no filas con valores NaN. Opcional. El valor predeterminado es Verdadero.

    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))    # Aplicas los pasos hacia atras 
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))   # Aplicas los pasos hacia adelante
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)    #Eliminamos las filas Nulas de nuestro df generado(Porque no sirven en la serie temporal)
    return agg

In [0]:
# Crear los datasets de entrenamiento, prueba y validación y verificar sus tamaños
numero_pasos_atras = 0  # Hiperparámetro
numero_pasos_futuro = 2   # Modelo uni-step igual a 1 paso,   MUltivariado seria mayor a 1 paso al futuro

# tr_s = series_to_supervised(tr.values, numero_pasos_atras , numero_pasos_futuro)
# vl_s = series_to_supervised(vl.values, numero_pasos_atras, numero_pasos_futuro)
# ts_s = series_to_supervised(ts.values, numero_pasos_atras, numero_pasos_futuro)

datos_df_s = series_to_supervised(df_escalado.values, numero_pasos_atras, numero_pasos_futuro)

# Asignar nombres originales a las columnas
original_columns = df_escalado.columns

new_columns = []
for i in range(numero_pasos_atras, 0, -1):
    new_columns += [f'{col}(t-{i})' for col in original_columns]
for i in range(0, numero_pasos_futuro):
    if i == 0:
        new_columns += [f'{col}(t)' for col in original_columns]
    else:
        new_columns += [f'{col}(t+{i})' for col in original_columns]

datos_df_s.columns = new_columns
datos_df_s.head()

In [0]:
datos_df_s_filter=datos_df_s[['Engine Coolant Temperature(t)','Left Exhaust Temperature(t)','Fuel Consumption Rate(t+1)']]
datos_df_s_filter.head()

In [0]:
df_escalado.head()

### OPCION 2: Utilizar una funcion solo para retroceso(lags)

### Algoritmo XGB Time Series

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
FEATURES = datos_df_s_filter.columns.drop('Fuel Consumption Rate(t+1)')

x = datos_df_s_filter[FEATURES]
y = datos_df_s_filter['Fuel Consumption Rate(t+1)']

In [0]:
x.head()

In [0]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False, random_state=1)
#shuffle=False, los datos no se mezclarán y se dividirán #secuencialmente según el orden(ideal para time series)

# Generamos los datasets de train y test
train  = pd.concat([X_train,y_train], axis=1)
test  = pd.concat([X_test,y_test], axis=1)

In [0]:
xgb_model = xgb.XGBRegressor(n_estimators=200,max_depth=10,objective='reg:squarederror',reg_alpha=0.1) #reg_alpha>>regulazrizacion Lasso educir la complejidad del modelo y evitar #el sobreajuste
#eval_set>>monitorear el rendimiento del modelo en los datos de evaluación durante el #entrenamiento
#objective=>>reg:squarederror>>regresión

xgb_model.fit(X_train,y_train,eval_set=[(X_train,y_train), (X_test,y_test)],eval_metric=['rmse'],
              early_stopping_rounds=50,verbose=True)
#verbose=True >> Mostrar en pantalla las iteraciones
#'auc'>>AUC es una métrica que evalúa la capacidad del modelo para distinguir entre clases positivas y negativas
# 'error'>> cuanto se equivoca el modelo en predecir (100-error)
# logloss>>un logloss más bajo indica un mejor rendimiento del modelo.

Feature Importance

In [0]:
f1 = pd.DataFrame(data=xgb_model.feature_importances_,index=xgb_model.feature_names_in_,columns=['importantes/variables'])

f1.sort_values('importantes/variables').plot(kind='barh',title='Variables')
plt.show()

In [0]:
y_pred

In [0]:
y_test

In [0]:
import matplotlib.pyplot as plt

# Paso 1: Obtener las predicciones para el conjunto de prueba (X_test)
y_pred = xgb_model.predict(X_test)

# Paso 2: Crear el gráfico
plt.figure(figsize=(10, 6))

# Gráfico de la serie temporal original (valores reales)
plt.plot(y_test, label='Valores reales', color='blue')

# Gráfico de la serie temporal predicha por el modelo
plt.plot(y_pred, label='Valores predichos', color='red')

# Añadir etiquetas y título
plt.title('Serie Temporal: Valores Reales vs Predichos')
plt.xlabel('Índice de Tiempo')
plt.ylabel('Consumo de Combustible')  # Cambia según el nombre de tu variable objetivo

# Mostrar la leyenda
plt.legend()

# Mostrar el gráfico
plt.show()

In [0]:
import matplotlib.pyplot as plt

# Paso 1: Obtener las predicciones para el conjunto de prueba (X_test)
y_pred = xgb_model.predict(X_test)

# Asegúrate de que y_pred tenga el mismo índice que y_test
y_pred_series = pd.Series(y_pred, index=y_test.index)

# Paso 2: Crear el gráfico con el índice de tiempo
plt.figure(figsize=(10, 6))

# Gráfico de la serie temporal original (valores reales)
plt.plot(y_test.index, y_test, label='Valores reales', color='blue')

# Gráfico de la serie temporal predicha por el modelo
plt.plot(y_pred_series.index, y_pred_series, label='Valores predichos', color='red')

# Añadir etiquetas y título
plt.title('Serie Temporal: Valores Reales vs Predichos')
plt.xlabel('Fecha')  # O lo que sea que represente tu índice temporal
plt.ylabel('Consumo Combustible (Galones)')  # Cambia según tu variable objetivo

# Mostrar la leyenda
plt.legend()

# Mostrar el gráfico
plt.show()

In [0]:
import plotly.graph_objects as go

# Obtener las predicciones para el conjunto de prueba (X_test)
y_pred = xgb_model.predict(X_test)

# Crear una serie de pandas para las predicciones con el mismo índice que y_test
y_pred_series = pd.Series(y_pred, index=y_test.index)

# Crear el gráfico con Plotly
fig = go.Figure()

# Añadir la serie de valores reales
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='Valores Reales', line=dict(color='blue')))

# Añadir la serie de valores predichos
fig.add_trace(go.Scatter(x=y_pred_series.index, y=y_pred_series, mode='lines', name='Valores Predichos', line=dict(color='red')))

# Personalizar el gráfico
fig.update_layout(
    title='Serie Temporal: Valores Reales vs Predichos',
    xaxis_title='Fecha',
    yaxis_title='Consumo Combustible (Galones)',
    hovermode='x unified'
)

# Mostrar el gráfico
fig.show()

In [0]:
# from kafka import KafkaConsumer
# from kafka import KafkaProducer
# from langchain.document_loaders import WebBaseLoader